In [42]:
library(dplyr)

df <- read.csv("processed.cleveland.data", header = FALSE)

In [43]:
colnames(df) <- c(
  "age", "sex", "cp", "restbp", "chol", "fbs", "restecg", "thalach", 
  "exang", "oldpeak", "slope", "ca", "thal", "hd"
)

df[df == "?"] <- NA
df <- na.omit(df)

df <- as.data.frame(lapply(df, function(x) as.numeric(as.character(x))))
df$hd <- ifelse(df$hd != 0, 1, 0)

df$cp <- as.factor(df$cp)
df$restecg <- as.factor(df$restecg)
df$slope <- as.factor(df$slope)
df$thal <- as.factor(df$thal)
df$sex <- as.factor(df$sex)
df$fbs <- as.factor(df$fbs)
df$exang <- as.factor(df$exang)

dummies <- model.matrix(~ cp + restecg + slope + thal - 1, data = df)

df_final <- cbind(df[, c("age", "sex", "restbp", "chol", "fbs", "thalach", 
                         "exang", "oldpeak", "ca", "hd")], dummies)

cat("Dimensiones finales del dataset limpio:\n")
print(dim(df_final))
cat("\nVista previa:\n")
print(head(df_final))

## Implementación del programa

In [56]:
set.seed(123)
df <- df_final

# creamos T
df$T <- sample(c(0, 1), size = nrow(df), replace = TRUE)

# creamos el error aleatorio
eps <- rnorm(nrow(df), mean = 0, sd = 1)

df$Y <- (1 + 0.05 * df$age + 0.3 * as.numeric(as.character(df$sex)) + 
          0.2 * df$restbp) * df$T + 0.5 * df$oldpeak + eps

print(head(df[, c("age", "sex", "restbp", "oldpeak", "T", "Y")]))

In [62]:
# OLS
modelo_ols <- lm(Y ~ T, data = df)
ate <- coef(modelo_ols)["T"]
cat("\nEfecto estimado del tratamiento (ATE) por OLS:", round(ate, 4), "\n")

In [65]:
library(randomForest)

# Random forest
X <- subset(df, select = -c(Y, T))
set.seed(123)

rf_treated <- randomForest(
  x = X[df$T == 1, ],
  y = df$Y[df$T == 1],
  ntree = 100
)

rf_control <- randomForest(
  x = X[df$T == 0, ],
  y = df$Y[df$T == 0],
  ntree = 100
)

y_pred_treated <- predict(rf_treated, X)
y_pred_control <- predict(rf_control, X)

df$ITE <- y_pred_treated - y_pred_control
ATE <- mean(df$ITE)
cat("\nEfecto promedio estimado (ATE) con Random Forest:", round(ATE, 4), "\n")

In [72]:
png("histograma_efectos_individuales_R.png", width = 800, height = 500)
hist(
  df$ITE, breaks = 30, col = "skyblue", border = "black",
  main = "Distribución de efectos individuales (ITE) - Random Forest (R)",
  xlab = "Efecto del tratamiento estimado", ylab = "Frecuencia"
)
dev.off()

cat("\nHistograma guardado como 'histograma_efectos_individuales_R.png'\n")

In [74]:
# Arbol representativo
library(rpart)
library(rpart.plot)

set.seed(123)
tree_model <- rpart(ITE ~ ., data = df[, c(names(X), "ITE")],
                    control = rpart.control(maxdepth = 2))

png("arbol_representativo_R.png", width = 900, height = 600)
rpart.plot(tree_model,
           main = "Árbol representativo (max_depth = 2)",
           type = 2, extra = 101, fallen.leaves = TRUE,
           box.palette = "Blues", shadow.col = "gray")
dev.off()

cat("\nÁrbol representativo guardado como 'arbol_representativo_R.png'\n")

El árbol representativo evidencia que la variable presión arterial en reposo (restbp) es el principal factor que explica las diferencias en el efecto del tratamiento sobre la salud cardiovascular. Los individuos con restbp menor a 139 presentan, en promedio, un efecto del tratamiento más bajo (valor medio cercano a 29–30), mientras que aquellos con valores superiores tienden a obtener un efecto mayor (valor promedio ≈ 34–35). Dentro del grupo con presión más baja, quienes tienen restbp menor a 123 muestran el efecto más reducido (≈ 27), lo que sugiere que el tratamiento tiene un impacto más fuerte en personas con niveles de presión arterial inicial más altos. En conjunto, el árbol confirma que la presión arterial en reposo es el factor que más influye en la heterogeneidad del efecto del tratamiento.

In [76]:
# importancias de las variables
importancias <- tree_model$variable.importance
importancias <- sort(importancias, decreasing = TRUE)

importancias_df <- data.frame(
  Variable = names(importancias),
  Importancia = importancias
)

print(importancias_df)

png("importancia_variables_arbol_R.png", width = 800, height = 500)
barplot(
  importancias_df$Importancia,
  names.arg = importancias_df$Variable,
  las = 2, col = "skyblue", border = "black",
  main = "Importancia de las variables en el árbol (max_depth = 2)",
  ylab = "Importancia relativa"
)
dev.off()

cat("\nGráfico de importancias guardado como 'importancia_variables_arbol_R.png'\n")

El gráfico muestra que la variable presión arterial en reposo (restbp) es, con diferencia, la más importante dentro del árbol, lo que indica que fue el principal criterio de partición para explicar las variaciones en el efecto del tratamiento sobre la salud cardiovascular. Las demás variables, como edad (age), colesterol (chol) y el descenso del segmento ST (oldpeak), tienen una contribución menor, lo que sugiere que su influencia sobre el resultado es marginal en comparación con la presión arterial. En conjunto, los resultados confirman que restbp domina la estructura del modelo y determina la mayor parte de la heterogeneidad observada en los efectos del tratamiento.

In [80]:
library(scales)

X_vars <- df[, sapply(df, is.numeric)]
X_vars <- X_vars[, !(names(X_vars) %in% c("Y", "T", "ITE"))]
X_std <- as.data.frame(scale(X_vars))

df$tercil <- cut(
  df$ITE,
  breaks = quantile(df$ITE, probs = c(0, 1/3, 2/3, 1)),
  labels = c("bajo", "medio", "alto"),
  include.lowest = TRUE
)

mean_table <- aggregate(X_std, by = list(Tercil = df$tercil), FUN = mean)
rownames(mean_table) <- mean_table$Tercil
mean_table <- mean_table[, -1]

png("heatmap_terciles_R.png", width = 1000, height = 500)
heatmap(
  as.matrix(mean_table), Rowv = NA, Colv = NA, scale = "none",
  col = colorRampPalette(c("blue", "white", "red"))(100),
  margins = c(10, 6),
  main = "Distribución de covariables estandarizadas por terciles del efecto del tratamiento"
)
dev.off()

cat("\nHeatmap guardado como 'heatmap_terciles_R.png'\n")

El mapa de calor muestra cómo varían las covariables estandarizadas según el tercil del efecto del tratamiento estimado. Se observa que la variable presión arterial en reposo (restbp) tiene el mayor contraste: valores más altos (en rojo) se asocian con los individuos del tercil alto, mientras que valores más bajos (en azul) caracterizan al tercil bajo. Esto sugiere que la presión arterial juega un papel clave en la magnitud del efecto del tratamiento. Asimismo, variables como edad y colesterol (chol) presentan ligeras diferencias entre terciles, indicando que individuos de mayor edad y con colesterol más elevado tienden a concentrarse en los grupos con efectos más altos. En cambio, otras variables —como las dummies de diagnóstico y los factores de tipo de dolor de pecho o pendiente del ST— muestran menor variabilidad entre grupos, sugiriendo un impacto menos heterogéneo sobre el resultado.